## Exemple de clustering de GHM

Utilisation de *sentence_transformers* pour créer des clusters de GHM par similarité sémantique.

In [1]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
import csv
import time
import random as rd

In [2]:
from sentence_transformers import SentenceTransformer, util

# Sentence embeddings model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
#exemples de codes GHM

GHM= ["28z04z", "28z07z", "05m092",
      "02c05j", "28z14z", "05m093",
      "28z17z", "28z18z", "05k101",
      "05k051", "05m042", "08c322"
     ]

In [4]:
#creation d'une base de données fictives
db=[]
nb_patients=10

for i in range(nb_patients):
    db.append(rd.choices(GHM, k=rd.randint(3,6)))
db

[['05m042', '08c322', '28z07z', '05m093'],
 ['28z07z', '28z17z', '28z07z', '05m042', '28z07z', '28z04z'],
 ['05m042', '05k101', '28z17z', '28z18z', '28z14z', '05m092'],
 ['28z04z', '05m042', '05m042', '05m042', '08c322', '08c322'],
 ['28z04z', '05k051', '05k101', '05m093', '05m093', '05m042'],
 ['28z17z', '08c322', '08c322', '28z17z'],
 ['05m092', '28z07z', '05m092', '28z18z', '28z14z', '08c322'],
 ['05k051', '05m093', '05m042', '05m092'],
 ['05k051', '05m092', '08c322', '05m092', '05m093'],
 ['02c05j', '05m093', '28z04z', '28z17z']]

In [5]:
GHM

['28z04z',
 '28z07z',
 '05m092',
 '02c05j',
 '28z14z',
 '05m093',
 '28z17z',
 '28z18z',
 '05k101',
 '05k051',
 '05m042',
 '08c322']

In [6]:
corpus_sentences=GHM
type(corpus_sentences)

list

In [7]:
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus. This might take a while


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
corpus_embeddings.shape

torch.Size([12, 384])

In [9]:
clusters = util.community_detection(corpus_embeddings, min_community_size=1,
                                    threshold=0.8
                                   )

In [10]:
clusters

[[0, 1, 4, 6, 7], [2, 5, 8, 9, 10], [3], [11]]

In [11]:
nb_clusters=len(clusters)
df_cluster=pd.DataFrame(columns=['Cluster','GHM'])
j=0

for i, cluster in enumerate(clusters):
    print(i, cluster)
    cluster_len=len(cluster)
  
    for sentence_id in cluster[0:len(cluster)]:
        df_cluster.loc[j,['Cluster']]=i+1
        df_cluster.loc[j,['GHM']]=corpus_sentences[sentence_id]
        j=j+1

0 [0, 1, 4, 6, 7]
1 [2, 5, 8, 9, 10]
2 [3]
3 [11]


In [12]:
for i in range(1,len(clusters)+1):
    print("Cluster", i, ":", df_cluster[df_cluster['Cluster']==i]['GHM'].tolist())

Cluster 1 : ['28z04z', '28z07z', '28z14z', '28z17z', '28z18z']
Cluster 2 : ['05m092', '05m093', '05k101', '05k051', '05m042']
Cluster 3 : ['02c05j']
Cluster 4 : ['08c322']


In [13]:
df_cluster

,Cluster,GHM
0,1,28z04z
1,1,28z07z
2,1,28z14z
3,1,28z17z
4,1,28z18z
5,2,05m092
6,2,05m093
7,2,05k101
8,2,05k051
9,2,05m042
